In [1]:
#Task 1
#import the required module
import pandas as pd
import os
#Combine the data scraped from May 22, 2022
file1 = "C:/Users/27737/compsoc/indeed_scraped_data/job_info_data/parsehub_5222022_v1.csv"
file2 = "C:/Users/27737/compsoc/indeed_scraped_data/job_info_data/parsehub_5222022_v2.csv"
file3 = "C:/Users/27737/compsoc/indeed_scraped_data/job_info_data/parsehub_5222022_v3.csv"
file4 = "C:/Users/27737/compsoc/indeed_scraped_data/job_info_data/parsehub_5222022_v4.csv"
file5 = "C:/Users/27737/compsoc/indeed_scraped_data/job_info_data/parsehub_5222022_v5.csv"
file6 = "C:/Users/27737/compsoc/indeed_scraped_data/job_info_data/parsehub_5222022_v6.csv"
combined_df = pd.concat(
   map(pd.read_csv, [file1, file2,file3,file4,file5,file6]), ignore_index=True)

#drop the rows with missing value in job titles and/or job descriptions.
cleanedcombined_df = combined_df.dropna(subset=['lnks_job_title', 'lnks_job_description'])
cleanedcombined_df = cleanedcombined_df.drop_duplicates('lnks_link').reset_index(drop=True)
print(cleanedcombined_df.shape)

(3554, 6)


In [2]:
#Use `spacy` to tokenize all the job titles included in the cleaned dataframe. 
import spacy
import en_core_web_sm
from tqdm import tqdm
nlp = spacy.load('en_core_web_sm')

In [5]:
#For each job title, find all the nouns and all the adjectives in the title and get their lowercased lemmatized form.


token2tag = dict()
for i in tqdm(range(len(cleanedcombined_df))):
    job_title = cleanedcombined_df.loc[i,'lnks_job_title']
    for token in nlp(job_title):
        if token.pos_ == 'NOUN'or 'PROPN':
            token2tag[token.lemma_.lower()] = token2tag.get(token.lemma_.lower(), 0)+1
                        
vocab_list1 = list(token2tag.keys())
print(len(vocab_list1))

100%|█████████████████████████████████████████████████████████████████████████████| 3554/3554 [00:23<00:00, 150.06it/s]

2308


In [7]:
token2tag2 = dict()
for i in tqdm(range(len(cleanedcombined_df))):
    job_title = cleanedcombined_df.loc[i,'lnks_job_title']
    for token in nlp(job_title):
        if token.pos_ == 'ADJ':
            token2tag2[token.lemma_.lower()] = token2tag2.get(token.lemma_.lower(), 0)+1
                        
vocab_list2 = list(token2tag2.keys())
print(len(vocab_list2))

100%|█████████████████████████████████████████████████████████████████████████████| 3554/3554 [00:24<00:00, 146.11it/s]

78


There are 2308 unique nouns and 78 unique adjectives.

In [8]:
print(vocab_list1)

['certified', 'pharmacy', 'technician', 'ii', '-retail', 'information', 'security', 'advisor', 'associate', 'director', 'learning', 'management', 'systems', 'plant', 'engineer', 'medical', 'staff', 'coordinator', 'player', 'services', 'cashier', 'audit', 'ft', 'mortgage', 'loan', 'underwriter', 'safety', 'sr', 'project', 'manager', 'customer', 'rge', 'advisory', 'compliance', 'specialist', 'principal', 'quality', '–', 'commercial', 'utility', 'operator', 'mill', 'sanitation', '-', '$', '5,000', 'first', 'year', 'sign', 'on', 'bonus', 'senior', 'privacy', 'officer', 'osha', 'regulations', 'sme', 'operations', 'supervisor', 'technical', 'facility', 'sh&e', 'sterile', 'compounding', 'tech', '.', 'patient', 'access', 'concierge', 'st.', 'peter', "'s", 'hospital', 'day', 'him', 'team', 'member', '(', 'front', 'end', 'service', ')', 'part', 'time', 'physician', 'office', ',', 'women', 'health', 'sales', 'assurance', 'retirement', 'nurse', 'practitioner', 'full', 'weekend', 'reg', 'histology'

In [76]:
vocab_list2

['advisory',
 'commercial',
 'full',
 'associate',
 'senior',
 'out',
 'human',
 '3rd',
 'personal',
 'spsus',
 'surgical',
 'internal',
 'clinical',
 'dietitian',
 'residential',
 'paramedic',
 'north',
 'electrical',
 'part',
 'environmental',
 'technician',
 'corporate',
 'supervisory',
 'spanish',
 'regional',
 'whole',
 'financial',
 'mechanical',
 'direct',
 'payable',
 'clinic',
 'quantitative',
 'skilled',
 'top',
 'dietary',
 'legislative',
 'private',
 'microbiologist',
 'licensed',
 'global',
 'archaeological',
 'early',
 'operational',
 'advanced',
 'first',
 'practical',
 'long',
 'care-',
 'virtual',
 'regulatory',
 'gfcii',
 'procurement',
 'educational',
 'bio',
 '-',
 'medical',
 'bilingual',
 'available',
 'instructional',
 'new',
 'dental',
 'small',
 'sanitarian',
 'chief',
 'chha-',
 '5th',
 'mid',
 '2nd',
 'analyst',
 'real',
 'civil',
 'hybrid',
 'remote',
 'registered',
 'lab',
 'recent',
 'iosha',
 'heavy']

The nouns and pronuns tell more about what the jobs about, and the adjectives convey less information but highlight the language needed and remote or not.

In [10]:
#Task 2
#Choose the first job title in your dataframe as the primary string. 
#Use one-hot encoding as the word embedding method and find jobs in your cleaned dataframe that have similar nouns 
#in the title as your primary string. 

job_title = cleanedcombined_df.loc[0, 'lnks_job_title']
job_title

'Certified Pharmacy Technician II -Retail Pharmacy'

In [73]:
import numpy as np
one_hot_encodings = []
existent = []

for i in tqdm(range(len(cleanedcombined_df))):
    job_title = cleanedcombined_df.loc[i, 'lnks_job_title'] 
    token_indices = []

    for token in nlp(job_title):
        if token.pos_=='NOUN'or token.pos_=='PROPN':
            token_index_in_vocab = vocab_list1.index(token.lemma_.lower())
            token_indices.append(token_index_in_vocab)
                
    one_hot_encoding = np.zeros(len(vocab_list1))
    if token_indices:
        existent.append(i)
    for token_index in token_indices:
        one_hot_encoding[token_index] = 1

    one_hot_encodings.append(one_hot_encoding)

100%|█████████████████████████████████████████████████████████████████████████████| 3554/3554 [00:24<00:00, 146.00it/s]


In [59]:
title_a

'Certified Pharmacy Technician II -Retail Pharmacy'

In [68]:
similarity_values

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.25,
 0.16666666666666674,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.35355339059327373,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.35355339059327373,
 0.0,
 0.18898223650461365,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.25,
 0.0,
 0.0,
 0.0,
 0.22360679774997894,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.25,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.18898223650461365,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.28867513459481287,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.28867513459481287,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,

In [74]:
from scipy.spatial.distance import cosine

title_a = cleanedcombined_df.loc[0, 'lnks_job_title']
similarity_values = []
for i in range(1, len(cleanedcombined_df)):
    if i in existent:
        similarity_value = 1 - cosine(one_hot_encodings[0], one_hot_encodings[i])
        similarity_values.append(similarity_value)
    else:
        similarity_values.append(np.nan)

In [75]:
similar_df = pd.DataFrame(columns=['job_title', 'similarity_value_with_one_hot'])
similar_df['job_title'] = cleanedcombined_df.loc[1:, 'lnks_job_title']
similar_df['similarity_value_with_one_hot'] = similarity_values

similar_df.sort_values(by='similarity_value_with_one_hot', ascending=False)
similar_df.nlargest(15, 'similarity_value_with_one_hot')

,job_title,similarity_value_with_one_hot
437,Pharmacy Technician,0.707107
471,Pharmacy Technician,0.707107
774,Pharmacy Technician,0.707107
800,Pharmacy Technician,0.707107
824,Pharmacy Technician,0.707107
1006,Nationally Certified Pharmacy Technician,0.707107
1858,Pharmacy Technician,0.707107
1908,Pharmacy Technician,0.707107
2269,Pharmacy Technician,0.707107
2907,Pharmacy Technician,0.707107


Task 3. Use spacy's word vector to do Task 2. Compare the results.

In [95]:
vector_indices = []
eligible_titles = []

for i in range(len(cleanedcombined_df)):
    spacy_encoding = np.array([float(0) for i in range(96)])
    token_num = 0
    for token in nlp(cleanedcombined_df.loc[i, 'lnks_job_title']):
        if token.pos_ == 'NOUN' or token.pos_ == 'PROPN':
            token_num += 1
            spacy_encoding += np.array(token.vector)
    if token_num != 0:
        spacy_encoding /= token_num
        vector_indices.append(spacy_encoding)
        eligible_titles.append(i)
        
similar_wordvector = []

for i in range(1, len(cleanedcombined_df)):
    if i in eligible_titles:
        similar_wordvector.append(1 - cosine(vector_indices[0],vector_indices[eligible_titles.index(i)]))
    else:
        similar_wordvector.append(np.nan)

similar_wordvector_df = pd.DataFrame(columns= ['job_titles','similarity_value_with_wordvectors'])
similar_wordvector_df['job_titles']=cleanedcombined_df.loc[1:,'lnks_job_title']
similar_wordvector_df['similarity_value_with_wordvectors'] = similar_wordvector 



In [96]:
similar_wordvector_df

,job_titles,similarity_value_with_wordvectors
1,Information Security Advisor,0.735964
2,Associate Director Learning Management Systems,0.762792
3,Plant Engineer,0.566088
4,Medical Staff Coordinator,0.807394
5,Player Services Cashier,0.684655
...,...,...
3549,Aviation Ground Support Technician,0.757933
3550,Loan Document Imaging Specialist,0.767176
3551,Field Tech,0.539486
3552,Associate Director-Operations,0.670419


The results came from word vector are more nuanced than one-hot encoding.